In [1]:
cd C:\Users\USER\Desktop\승호\3학년 2학기(2018)\정보검색과 데이터마이닝\과제\hw_4 - 문서 유사도 측정

C:\Users\USER\Desktop\승호\3학년 2학기(2018)\정보검색과 데이터마이닝\과제\hw_4 - 문서 유사도 측정


### fnames 파일 읽어서 list 형태로 가지고 있자

In [2]:
import subprocess
import os

f = open("fnames.txt", 'r')
f_list = []
while True:
    line = f.readline()
    if not line: break
    f_list.append(line[:-1])
f.close()
# f_list

In [3]:
cd klt2010/EXE

C:\Users\USER\Desktop\승호\3학년 2학기(2018)\정보검색과 데이터마이닝\과제\hw_4 - 문서 유사도 측정\klt2010\EXE


### 1-1) index2018.exe를 이용하여 각 텍스트 파일에 대한 인덱싱 파일을 만들자(색인어 추출)
* 추가로 각 파일당 unique한 색인어만 들어간 all_unique_indexing.txt파일 만들기

In [4]:

# all_unique_indexing파일 존재하면 삭제
if os.path.exists("all_unique_indexing.txt"):
    os.remove("all_unique_indexing.txt")
f = open("all_unique_indexing.txt", 'a')

# 색인어파일 디렉토리 있으면 삭제
if os.path.exists("색인어파일"):
    subprocess.call("rm -rf 색인어파일")
subprocess.call("mkdir 색인어파일")

for i in range(len(f_list)):
    input_f_name = "../../textfiles/" + f_list[i]
    output_f_name = "./색인어파일/" + str(i) + ".txt"
        
    # 각 파일 indexing file 만듬
    cmd_exe = " ".join(["index2018.exe", input_f_name, output_f_name])
    subprocess.call(cmd_exe)
    
    # 각 파일에 unique한 색인어 목록 all_unique_indexing.txt파일에 써주기
    cmd_exe = " ".join(["wordcount.exe -new -uniq -i", output_f_name, "temp_unique_indexing.txt"])
    subprocess.call(cmd_exe)
    f_read = open("temp_unique_indexing.txt", 'r', encoding = "ANSI")
    lines = f_read.readlines()
    for line in lines:
        try:
            f.write(line)
        except:
            print("encoding error")
    f_read.close()

f.close()    

### 1-2) 만든 인덱싱 파일과 wordcount.exe를 이용해서 모든 파일에 docid : (빈도수, 단어), (빈도수, 단어) 형태로 저장하자

In [5]:
forward_indexing_table = dict()

for i in range(len(f_list)):

    max_tf = 0
    
    # word_count실행
    f_name = "./색인어파일/" + str(i) + ".txt"
    cmd_exe = " ".join(["wordcount.exe -new -i", f_name, f_name])
    subprocess.call(cmd_exe)
    
    #파일 열어서 forward_indexing_table만들기
    f = open(f_name, "r", encoding = "ANSI")
    word_list = []
    while True:
        try:
            line = f.readline()
            if not line:
                break
            temp_list = line.split()
            word_list.append(temp_list)
            # 각 문서의 max_tf구하는 과정 후에 문서유사도에서 tf계산할 때 쓰임
            if max_tf < int(temp_list[0]):
                max_tf = int(temp_list[0])
        except:
            print("encoding - error")
    # key는 docid = i 고 value는 해당파일 word_list를 가진 dictionary로 만든다.
    word_list.append(max_tf)
    forward_indexing_table[i] = word_list.copy()
    
print(forward_indexing_table[2])

f.close()

[['2', '08'], ['1', '120Gb'], ['1', '1600'], ['1', '1Gb'], ['3', '20'], ['2', '2003'], ['1', '20일'], ['2', '21'], ['2', '23'], ['1', '23일'], ['2', '25'], ['1', '25일'], ['3', '32'], ['3', '32비트'], ['1', '37'], ['2', '3D'], ['1', '3D게'], ['1', '3D게임'], ['1', '46'], ['24', '64'], ['16', '64비트'], ['1', '64비트급'], ['13', 'AMD'], ['6', 'CPU'], ['1', 'CPU개발계획'], ['1', 'DDR'], ['1', 'HDD'], ['2', 'HP'], ['2', 'IBM'], ['1', 'IBM도'], ['24', 'PC'], ['1', 'PC기종'], ['1', 'PC수요'], ['2', 'PC시장'], ['3', 'PC업체'], ['3', 'PC용'], ['1', 'XP'], ['1', 'bailh'], ['1', 'bailh@etnews.co.kr'], ['1', 'co'], ['1', 'etnews'], ['1', 'kr'], ['1', '가능성'], ['1', '감지'], ['1', '강조'], ['4', '개발'], ['1', '개발자'], ['1', '개발중'], ['1', '개선'], ['1', '거'], ['2', '것'], ['1', '게'], ['2', '게임'], ['1', '게임전문가'], ['1', '게재'], ['1', '결정'], ['1', '결정키'], ['2', '계획'], ['3', '고'], ['1', '고객'], ['1', '고성능'], ['1', '고성능PC'], ['1', '고속'], ['1', '고위'], ['1', '공동'], ['6', '관계자'], ['1', '관련'], ['1', '관심'], ['1', '관심도'], ['1', '구도'], ['2', '국내']

### 1-3) wordcount.exe를 all_unique_indexing.txt(각 파일당 unique indexing list)에 이용하여 DF계산하자

In [6]:
output_f_name = "word_df.txt"
cmd_exe = " ".join(["wordcount.exe -new -i","all_unique_indexing.txt", output_f_name])
subprocess.call(cmd_exe)

# word_df 파일을 wordcount하여서 각 색인어당 df계산 [색인어, df]
f = open("word_df.txt", 'r')
lines = f.readlines()
df_table = dict()
for line in lines:
    try:
        a = line.split()
        df_table[a[1]] = a[0] 
    except:
        pass
# print(df_table)

### 1-4) <TID, DF> table 구성 (그냥 색인어 대신 TID를 넣어주면 된다.)

In [7]:
tid_df_table = []
index = 0
for df in df_table.values():
    tid_df_table.append((index, df))
    index = index + 1
# print(tid_df_table)

### 문서 유사도 계산을 위해 docid : [ (term, weight) , (term, weight)  ... ] 형태로 저장
### weight = tf * idf
### tf = 문서에서 단어 나온 횟수 / 문서에서 제일 많이 나온 단어횟수
### idf = log(전체문서개수 / df(단어가 문서전체에 나온 횟수)) 

In [8]:
import math

doc_vector = dict()

# forward_indexing_table을 이용하여 docid와 term과 빈도수가 담긴 튜플들을 가져와 weight를 계산하여 다시 저장한다.
for docid, tid_tf_tuples in forward_indexing_table.items():
    
    word_list = []
    
    # 마지막은 max_tf니까 마지막 전까지 루프를 돌린다
    for tuple in tid_tf_tuples[:-1]:
        
        # tuple[0]에는 빈도수, tid_tf_tuples[-1]는 max_tf 
        tf = int(tuple[0]) / tid_tf_tuples[-1]
        
        # int(df_table[tuple[1]])는 해당 단어의 df
        idf = math.log(623 / int(df_table[tuple[1]]))
        
        weight = tf * idf
        word_list.append([tuple[1], weight])
        
    doc_vector[docid] = word_list.copy()

#docid가 2인 문서의 벡터를 보여주겠다. 
print(doc_vector[2])

[['08', 0.16405320001107243], ['120Gb', 0.2681061049494772], ['1600', 0.19344946039830824], ['1Gb', 0.2681061049494772], ['20', 0.1978145318584795], ['2003', 0.16797549258257127], ['20일', 0.1393126693928807], ['21', 0.15672246893224268], ['23', 0.2007662356710253], ['23일', 0.18146270737948403], ['25', 0.17422842474448078], ['25일', 0.14542114748420887], ['32', 0.3672548696651216], ['32비트', 0.6031385757941691], ['37', 0.15527067990355178], ['3D', 0.2953478967409407], ['3D게', 0.23922497242614615], ['3D게임', 0.23922497242614615], ['46', 0.14328392688472757], ['64', 4.036651245989082], ['64비트', 2.9034033180717445], ['64비트급', 0.2681061049494772], ['AMD', 2.0960317957182046], ['CPU', 0.915489449136918], ['CPU개발계획', 0.2681061049494772], ['DDR', 0.1645683278749772], ['HDD', 0.15527067990355178], ['HP', 0.21189385172306888], ['IBM', 0.19095098269966004], ['IBM도', 0.20104619193138967], ['PC', 1.5145655929593282], ['PC기종', 0.2681061049494772], ['PC수요', 0.2681061049494772], ['PC시장', 0.33638760383242

## 코사인유사도를 이용하여 doc_index를 증가시키며  upper triangle 구성

In [9]:
doc_index = 0

similarity_matrix = []

# 623개를 돌려야한다
for a_index, weight_tuples in doc_vector.items():
    
    # 오래걸려서 몇번 문서까지 진행되는지 확인하는 프린트문
    print(a_index)
    
    # 기준이 되는 문서의 weight의 합을 구하자! : 첫번째 분모 = a_denominator
    a_denominator = 0
    for tuple in weight_tuples:
        a_denominator += math.pow(tuple[1], 2)
    
    # doc_index를 증가시키며 upper triangle형태로 만든다
    temp_similarity = []
    for b_index in range(doc_index, 623):
        b_denominator = 0
        # 비교할 문서의 weight의 합을 구하자! : 두번째 분모 b_denominator
        for tuple in doc_vector[b_index]:
            b_denominator += math.pow(tuple[1], 2)
        # a * b 로 분모 구성 완료 
        denominator = math.sqrt(a_denominator) * math.sqrt(b_denominator)
        
        # 각 term의 weight곱의 합을 구해야한다!
        numerator = 0
        for a_tuple in weight_tuples:
            for b_tuple in doc_vector[b_index]:
                if a_tuple[0] == b_tuple[0]:
                    numerator += a_tuple[1] * b_tuple[1]
#                    print(a_tuple, b_tuple)
                    break
        similarity = numerator / denominator
        temp_similarity.append(similarity)
    
#     doc_index += 1
    similarity_matrix.append(temp_similarity.copy())
    
print(similarity_matrix[2])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
cd C:\Users\USER\Desktop\승호\3학년 2학기(2018)\정보검색과 데이터마이닝\과제\hw_4 - 문서 유사도 측정

C:\Users\USER\Desktop\승호\3학년 2학기(2018)\정보검색과 데이터마이닝\과제\hw_4 - 문서 유사도 측정


### 결과를 csv 파일에 쓰기
### 623 X 623 매트릭스를 print하기 힘들어 csv파일에 써주겠다.

In [14]:
import csv

f = open('output.csv', 'w', newline='')
csv_writer = csv.writer(f)

zero_count = 0
for similarity_vector in similarity_matrix:
    # 0번인덱스부터 채웠기 때문에 upper triangle로 만들어줄려면 앞에 0으로 채워줘야한다.
    row = ([0] * zero_count) + (similarity_vector[zero_count:])
#     row = similarity_vector
    csv_writer.writerow(row)
    zero_count += 1
f.close()

### 임의의 문서 x 와 제일 유사도가 높은 문서는?!

In [22]:
x = int(input('임의의 문서 x 숫자를 입력해주세요. (0 ~ 622)'))

# x를 제외하고 max를 찾는 이유는 x index에는 자기 자신과의 유사도라 무조건 max이기 때문이다.
similarity_value = max(similarity_matrix[x][:x] + similarity_matrix[x][x+1:])

# similarity_value로 index를 찾아 doc_number를 찾는다
doc_number = similarity_matrix[x].index(similarity_value)

print('가장 유사도가 높은 문서번호 : ', doc_number ,', 유사도 : ', similarity_value)

# 처음에 만들었는 file name list를 활용하여 예쁘게 출력해본다.
print(f_list[x], '와 제일 유사도가 높은 문서는 ', f_list[doc_index], '이다.')

임의의 문서 x 숫자를 입력해주세요. (0 ~ 622)8
가장 유사도가 높은 문서번호 :  9 , 유사도 :  0.5472360332114795
IT-baidu.txt 와 제일 유사도가 높은 문서는  IT-2007newWebIR.txt 이다.


과제를 통하여 간단하게 문서유사도 계산을 만들었다. 분석하는 재미가 있었다. 하지만 실제로 유사도 계산 프로그램을 만들면 이렇게 만들면 안될 것 같다. 개선해야 할 점들을 정리해보겟다.

개선해야할 점

term말고 tid를 활용하여 저장공간과 효율적인 접근 가능하게 하기

tf의 정의가 여러가지 있음.
1. 그냥 빈도수,
2. 빈도수 / 문서의 전체 빈도수의 합
3. 빈도수 / 문서에서 가장 큰 빈도수
4. 로그변환 드등 
이거에 따라 유사도 값이 달라지는 것을 알아야 할듯...

623개의 문서뿐만 아니라 어떤 데이터 셋이 txt파일로 와도 될수 있게 바꾸기
ex) 623 -> len(f_list)등등

loop횟수 줄이기 - 효율성, 알고리즘적 측면
weight를 구하는 부분에서 너무오래걸린다.
딕셔너리를 ordered되게해서 접근이 쉽게 만들기
서로 공통된 단어들을 갖고있는 벡터를 만들어 weight계산을 빠르게 만들어주기
각 문서의 weight들의 합 즉, 분모를 구성하는 걸 전에 loop돌릴 때 계산해서 값을 미리 저장해 놓는 것!
등등 시간복잡도를 개선해야 할 부분들이 많이 보였다.

623개의 문서만 하여 ram이 부족하진 않았지만, 실제로 큰 데이터 셋을 마주하게 되면
file i/o를 통하여 프로그램을 만들어야 할 것 같다.
그래서 csv파일 출력을 시도해 보았다.

교수님께서 제공해주신 wordcount, index 실행파일도 좋지만, scikit learn 알고리즘이나 konlpy등 
공개된 라이브러리를 이용하여 stemming, stopword처리부분을 활용하면 내가 원하는 결과를 얻을 수 있을 것 같다.

나중에 텍스트 분석이나 검색엔진 구현작업을 하게 되면 도움이 될 수 있는 유용한 과제였다.

In [ ]:
ma